In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
np.random.seed(2498)


columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
len(dataset.item_id.unique()),dataset.item_id.unique().min(),dataset.item_id.unique().max()

(24507, 0, 24506)

In [3]:
from Data_manager.Build_Matrix.Data_import import build_ICM_type,build_ICM_lengh,build_URM_ICM_onlyURM_item
import scipy.sparse as sps
URM_ALL,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
ICM_TYPE   =build_ICM_type(dataset_t)
ICM_LENGHT =build_ICM_lengh(dataset_l)
ICM_genre_subgenre = sps.hstack((ICM_TYPE, ICM_LENGHT))

In [4]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_ALL, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])


EvaluatorHoldout: Ignoring 697 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 308 ( 0.7%) Users that have less than 1 test interactions


In [5]:
import scipy.sparse as sps
stacked_URM_1 = sps.vstack([URM_train,ICM_genre_subgenre.T])
stacked_URM_3 = sps.vstack([URM_train, ICM_TYPE.T, ICM_TYPE.T])

In [6]:
import os

output_folder_path = "result_experiments_SLIMElasticNetRecommender_tuning_best"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 400
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
        "l1_ratio": Real(low = 1e-5, high = 0.1, prior = 'log-uniform'),
        "topK": Integer(200, 6000),
        "alpha": Real(low = 1e-5, high = 0.1, prior = 'log-uniform'),
        "workers": Categorical([6]),
    }


recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [stacked_URM_3],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = MultiThreadSLIM_SLIMElasticNetRecommender


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [7]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

SearchBayesianSkopt: Resuming 'SLIMElasticNetRecommender'... Loaded 145 configurations.
Iteration No: 1 started. Evaluating function at random point.


  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.8340
Function value obtained: -0.0216
Current minimum: -0.0224
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.0001370617592354718, 'topK': 4430, 'alpha': 0.09826178801566772, 'workers': 6}


100%|█████████▉| 24504/24507 [16:58<00:00, 38.34it/s]

EvaluatorHoldout: Processed 40932 (100.0%) in 53.97 sec. Users per second: 758
SearchBayesianSkopt: Config 145 is suboptimal. Config: {'l1_ratio': 0.0001370617592354718, 'topK': 4430, 'alpha': 0.09826178801566772, 'workers': 6} - results: PRECISION: 0.0402375, PRECISION_RECALL_MIN_DEN: 0.0657469, RECALL: 0.0611745, MAP: 0.0177523, MAP_MIN_DEN: 0.0287735, MRR: 0.1309423, NDCG: 0.0616377, F1: 0.0485447, HIT_RATE: 0.2943663, ARHR_ALL_HITS: 0.1518051, NOVELTY: 0.0038911, AVERAGE_POPULARITY: 0.3211410, DIVERSITY_MEAN_INTER_LIST: 0.8123955, DIVERSITY_HERFINDAHL: 0.9812376, COVERAGE_ITEM: 0.1201289, COVERAGE_ITEM_HIT: 0.0208104, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.2894377, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0060660, SHANNON_ENTROPY: 7.0261137, RATIO_DIVERSITY_HERFINDAHL: 0.9815425, RATIO_DIVERSITY_GINI: 0.0116249, RATIO_SHANNON_ENTROPY: 0.5185321, RATIO_AVERAGE_POPULARITY: 6.2885786, RATIO_NOVELTY: 0.2833392, 



...

100%|█████████▉| 24504/24507 [15:33:45<00:00, 48.30it/s]      

EvaluatorHoldout: Processed 40932 (100.0%) in 40.13 sec. Users per second: 1020
SearchBayesianSkopt: Config 242 is suboptimal. Config: {'l1_ratio': 0.012235177303577425, 'topK': 4588, 'alpha': 4.6191179481366176e-05, 'workers': 6} - results: PRECISION: 0.0441684, PRECISION_RECALL_MIN_DEN: 0.0686052, RECALL: 0.0631317, MAP: 0.0205515, MAP_MIN_DEN: 0.0319096, MRR: 0.1434372, NDCG: 0.0669115, F1: 0.0519743, HIT_RATE: 0.3044073, ARHR_ALL_HITS: 0.1708855, NOVELTY: 0.0044396, AVERAGE_POPULARITY: 0.1892834, DIVERSITY_MEAN_INTER_LIST: 0.9493022, DIVERSITY_HERFINDAHL: 0.9949279, COVERAGE_ITEM: 0.4657853, COVERAGE_ITEM_HIT: 0.0583507, ITEMS_IN_GT: 0.9953891, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.2993106, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0464974, SHANNON_ENTROPY: 9.6553352, RATIO_DIVERSITY_HERFINDAHL: 0.9952371, RATIO_DIVERSITY_GINI: 0.0891071, RATIO_SHANNON_ENTROPY: 0.7125706, RATIO_AVERAGE_POPULARITY: 3.7065452, RATIO_NOVELTY: 0.3232800, 



  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 99 ended. Evaluation done at random point.
Time taken: 56062.7246
Function value obtained: -0.0206
Current minimum: -0.0224
Iteration No: 100 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.0002146393728280065, 'topK': 2547, 'alpha': 0.04910998124878519, 'workers': 6}


 92%|█████████▏| 22664/24507 [15:21<01:07, 27.27it/s]

In [ ]:
from Recommenders.DataIO import DataIO
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import pandas as pd
recommender_class = SLIMElasticNetRecommender

output_folder_path = "result_experiments"
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
time_df = search_metadata["time_df"]

print('mean-time:')
print(time_df['train'].mean())


result_on_test_df = search_metadata["result_on_test_df"]
result_best_on_test = search_metadata["result_on_last"]
print('BEST_HYPERPARAMETERS: ')
print(search_metadata["hyperparameters_best"])
print(pd.DataFrame(result_on_test_df))
print(result_best_on_test)